In [4]:
### 2 input xor이 아닌 3 input xor로 변경 은닉층의 갯수에 따라 결과가 달라짐
import torch

device = 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

X = torch.FloatTensor([[0, 0,0], [0, 0,1], [0, 1,0], [0,1, 1],[1, 0,0],[1, 0,1],[1, 1,0],[1, 1,1],]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0],[1], [0], [0], [1]]).to(device)

linear1 = torch.nn.Linear(3,3,bias=True)
linear2 = torch.nn.Linear(3,1,bias=True)
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())

0 0.7439110279083252
100 0.6932504177093506
200 0.6932159066200256
300 0.6931926012039185
400 0.6931757926940918
500 0.6931623220443726
600 0.6931504607200623
700 0.6931392550468445
800 0.6931275725364685
900 0.6931143999099731
1000 0.6930986642837524
1100 0.6930789947509766
1200 0.6930529475212097
1300 0.6930175423622131
1400 0.692966639995575
1500 0.6928892135620117
1600 0.6927623748779297
1700 0.692533552646637
1800 0.692061722278595
1900 0.6908833980560303
2000 0.6869640350341797
2100 0.6682999134063721
2200 0.584000289440155
2300 0.4193096160888672
2400 0.20987842977046967
2500 0.09931641072034836
2600 0.05965084210038185
2700 0.04158172011375427
2800 0.031611863523721695
2900 0.025387229397892952
3000 0.021163541823625565
3100 0.018122756853699684
3200 0.01583496481180191
3300 0.014054317027330399
3400 0.012630550190806389
3500 0.01146705076098442
3600 0.010498893447220325
3700 0.009681030176579952
3800 0.008981173858046532
3900 0.008375605568289757
4000 0.007846543565392494
4100

In [8]:
import torch

learning_rate = 0.1

device = 'cpu'
torch.manual_seed(777)

X = torch.FloatTensor([[ 0,0], [ 0,1], [ 1,0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

w1 = torch.Tensor(2,2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2,1).to(device)
b2 = torch.Tensor(1).to(device)

def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))

for step in range(10001):
    l1 = torch.add(torch.matmul(X,w1),b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1,w2),b2)
    Y_pred = sigmoid(l2)

    cost = -torch.mean(Y*torch.log(Y_pred)+(1-Y)*torch.log(1-Y_pred))

    d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)

    d_l2 = d_Y_pred * sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1,0,1),d_b2)

    d_a1 = torch.matmul(d_b2, torch.transpose(w2, 0, 1))
    d_l1 = d_a1 * sigmoid_prime(l1)
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X,0,1), d_b1)

    w1 = w1 - learning_rate *d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1,0)
    w2 = w2 - learning_rate *d_w2
    b2 = b2 - learning_rate * torch.mean(d_b2,0)

    if step%100 == 0:
        print(step, cost.item())

with torch.no_grad():
    l1 = torch.add(torch.matmul(X,w1),b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1,w2),b2)
    hypothesis = sigmoid(l2)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())

0 2.1997134685516357
100 0.6431387066841125
200 0.5664221048355103
300 0.5105169415473938
400 0.46477317810058594
500 0.4242010712623596
600 0.3850454092025757
700 0.3463992774486542
800 0.30894935131073
900 0.27389979362487793
1000 0.24227359890937805
1100 0.21456950902938843
1200 0.19077196717262268
1300 0.17054003477096558
1400 0.15339647233486176
1500 0.13885094225406647
1600 0.12645864486694336
1700 0.11584067344665527
1800 0.10668490827083588
1900 0.09873753786087036
2000 0.09179384261369705
2100 0.08568848669528961
2200 0.08028796315193176
2300 0.07548407465219498
2400 0.07118801772594452
2500 0.06732720881700516
2600 0.06384152173995972
2700 0.060681067407131195
2800 0.05780404061079025
2900 0.05517538636922836
3000 0.05276529863476753
3100 0.05054834485054016
3200 0.04850300773978233
3300 0.04661067575216293
3400 0.04485523700714111
3500 0.043222662061452866
3600 0.04170087352395058
3700 0.04027919843792915
3800 0.038948263972997665
3900 0.037699975073337555
4000 0.03652677685

In [1]:
### 2 input xor이 아닌 3 input xor로 변경 은닉층의 갯수에 따라 결과가 달라짐
### 은닉층이 너무 많은경우 오히려 결과가 제대로 나오지 않음## 2 input xor이 아닌 3 input xor로 변경 은닉층의 갯수에 따라 결과가 달라짐
### 은닉층이 너무 많은경우 오히려 결과가 제대로 나오지 않음
import torch

device = 'cpu'

torch.manual_seed(777)

X = torch.FloatTensor([[0, 0,0], [0, 0,1], [0, 1,0], [0,1, 1],[1, 0,0],[1, 0,1],[1, 1,0],[1, 1,1],]).to(device)

import torch

device = 'cpu'

torch.manual_seed(777)

X = torch.FloatTensor([[0, 0,0], [0, 0,1], [0, 1,0], [0,1, 1],[1, 0,0],[1, 0,1],[1, 1,0],[1, 1,1],]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0],[1], [0], [0], [1]]).to(device)

linear1 = torch.nn.Linear(3,10,bias=True)
linear2 = torch.nn.Linear(10,10,bias=True)
linear3 = torch.nn.Linear(10,1,bias=True)
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=1)

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())

0 0.7012462019920349
100 0.6931397914886475
200 0.6931383013725281
300 0.6931366920471191
400 0.693135142326355
500 0.6931335926055908
600 0.6931319236755371
700 0.6931301355361938
800 0.6931282877922058
900 0.6931264400482178
1000 0.6931244134902954
1100 0.6931223273277283
1200 0.6931200623512268
1300 0.6931177973747253
1400 0.693115234375
1500 0.6931125521659851
1600 0.6931096315383911
1700 0.6931065917015076
1800 0.6931031942367554
1900 0.6930996179580688
2000 0.6930956840515137
2100 0.6930913329124451
2200 0.6930866241455078
2300 0.6930813789367676
2400 0.6930755376815796
2500 0.6930691003799438
2600 0.6930618286132812
2700 0.693053662776947
2800 0.6930443644523621
2900 0.6930336952209473
3000 0.6930214762687683
3100 0.6930070519447327
3200 0.692990243434906
3300 0.6929700970649719
3400 0.6929457783699036
3500 0.6929160356521606
3600 0.6928787231445312
3700 0.6928312182426453
3800 0.6927690505981445
3900 0.6926849484443665
4000 0.6925672888755798
4100 0.6923938989639282
4200 0.6921